In [1]:
import os
import shutil
import pandas as pd
import pydicom
from PIL import Image, ImageDraw
import matplotlib.pyplot as plt
import numpy as np

In [2]:
# Path to the Excel file containing the MRI coordinates
targeted_bx_path = '/Users/arjunmoorthy/Desktop/Research_Capstone/targeted_biopsy_patients.xlsx'

# Directory containing the DICOM images
dicom_dir_path = '/Users/arjunmoorthy/Desktop/targeted_bx_dicoms'

# Load the Excel file
targeted_df = pd.read_excel(targeted_bx_path)

In [3]:
# Checking to see that ImagePositionPatient Tag is extractable

dicom_file_path = '/Users/arjunmoorthy/Desktop/targeted_bx_dicoms/Prostate-MRI-US-Biopsy-0005/08-01-2007-NA-MRI PROSTATE W WO CONTRAST-06267/5.000000-t2spcrstaxial oblProstate-24293/1-02.dcm'
# Read the DICOM file
ds = pydicom.dcmread(dicom_file_path)

# Check if 'ImagePositionPatient' tag is present and extract the z-coordinate
if 'ImagePositionPatient' in ds:
    z_coord = ds.ImagePositionPatient[2]
    print(f"Z-Coordinate of the DICOM slice: {z_coord}")
else:
    print("The 'ImagePositionPatient' tag is not present in this DICOM file.")

Z-Coordinate of the DICOM slice: -33.064769983292


In [4]:
patient_id = 'Prostate-MRI-US-Biopsy-0003'  # Test Patient ID

# Path to the patient's DICOM directory
patient_dicom_dir = os.path.join(dicom_dir_path, patient_id)
print(f"Searching in: {patient_dicom_dir}")  

z_coords = []

# Walk through all subdirectories in the patient's DICOM directory
for root, dirs, files in os.walk(patient_dicom_dir):
    for file in files:
        if file.endswith(".dcm"):
            file_path = os.path.join(root, file)
            print(f"Reading DICOM: {file_path}")  
            ds = pydicom.dcmread(file_path)

            # Extract the z-coordinate
            if 'ImagePositionPatient' in ds:
                dicom_z_coord = ds.ImagePositionPatient[2]
                z_coords.append(round(dicom_z_coord, 2)) 
                print(f"Found z-coordinate: {dicom_z_coord}")  

# Check if any z-coordinates were found and print them
if z_coords:
    print(f"Z-coordinates for patient {patient_id}: {z_coords}")
else:
    print("No z-coordinates found.")


Searching in: /Users/arjunmoorthy/Desktop/targeted_bx_dicoms/Prostate-MRI-US-Biopsy-0003
Reading DICOM: /Users/arjunmoorthy/Desktop/targeted_bx_dicoms/Prostate-MRI-US-Biopsy-0003/03-22-2009-NA-MRI PROSTATE W WO CONTRAST-74733/13.000000-t2spcrstaxial oblProstate-44248/1-42.dcm
Found z-coordinate: 83.257481113076
Reading DICOM: /Users/arjunmoorthy/Desktop/targeted_bx_dicoms/Prostate-MRI-US-Biopsy-0003/03-22-2009-NA-MRI PROSTATE W WO CONTRAST-74733/13.000000-t2spcrstaxial oblProstate-44248/1-56.dcm
Found z-coordinate: 104.25748111308
Reading DICOM: /Users/arjunmoorthy/Desktop/targeted_bx_dicoms/Prostate-MRI-US-Biopsy-0003/03-22-2009-NA-MRI PROSTATE W WO CONTRAST-74733/13.000000-t2spcrstaxial oblProstate-44248/1-57.dcm
Found z-coordinate: 105.75748111308
Reading DICOM: /Users/arjunmoorthy/Desktop/targeted_bx_dicoms/Prostate-MRI-US-Biopsy-0003/03-22-2009-NA-MRI PROSTATE W WO CONTRAST-74733/13.000000-t2spcrstaxial oblProstate-44248/1-43.dcm
Found z-coordinate: 84.757481113076
Reading DICOM: 

In [ ]:
# Paths
output_dir_path = os.path.join(dicom_dir_path, 'targeted_bx_slices')

# Ensure the output directory exists
os.makedirs(output_dir_path, exist_ok=True)

# Load the Excel file
df = pd.read_excel(targeted_bx_path)

# Function to find and copy DICOM slice closest to the given z-coordinate
def find_and_copy_closest_slice(patient_id, z_coord):
    patient_dicom_dir = os.path.join(dicom_dir_path, patient_id)
    closest_file_path = None
    min_diff = np.inf  

    # Walk through all subdirectories
    for root, dirs, files in os.walk(patient_dicom_dir):
        for file in files:
            if file.endswith(".dcm"):
                file_path = os.path.join(root, file)
                ds = pydicom.dcmread(file_path)
                
                if 'ImagePositionPatient' in ds:
                    dicom_z_coord = ds.ImagePositionPatient[2]
                    diff = abs(dicom_z_coord - z_coord)
                    
                    if diff < min_diff:
                        min_diff = diff
                        closest_file_path = file_path
    
    # If found, Copy the closest slice to the output directory
    if closest_file_path:
        output_patient_dir = os.path.join(output_dir_path, patient_id)
        os.makedirs(output_patient_dir, exist_ok=True)
        shutil.copy(closest_file_path, output_patient_dir)
        print(f"Copied: {closest_file_path} to {output_patient_dir}")

# Loop through each row in the DataFrame to process each patient
for index, row in df.iterrows():
    patient_id = row['Patient Number']  
    z_coord = row['Bx Tip Z (MRI Coord)']  
    find_and_copy_closest_slice(patient_id, z_coord)


In [10]:
import os
import numpy as np
import pandas as pd
import pydicom
import shutil

# Assuming dicom_dir_path and targeted_bx_path are defined
output_dir_path = os.path.join(dicom_dir_path, 'targeted_bx_slices_bothz')

# Ensure the output directory exists
os.makedirs(output_dir_path, exist_ok=True)

# Load the Excel file
df = pd.read_excel(targeted_bx_path)

# Function to find and copy DICOM slices closest to the given z-coordinates
def find_and_copy_closest_slices(patient_id, z_coords):
    patient_dicom_dir = os.path.join(dicom_dir_path, patient_id)
    closest_file_paths = []
    min_diffs = [np.inf, np.inf]

    # Walk through all subdirectories
    for root, dirs, files in os.walk(patient_dicom_dir):
        for file in files:
            if file.endswith(".dcm"):
                file_path = os.path.join(root, file)
                ds = pydicom.dcmread(file_path)
                
                if 'ImagePositionPatient' in ds:
                    dicom_z_coord = ds.ImagePositionPatient[2]
                    
                    for i, z_coord in enumerate(z_coords):
                        diff = abs(dicom_z_coord - z_coord)
                        
                        if diff < min_diffs[i]:
                            min_diffs[i] = diff
                            if len(closest_file_paths) > i:
                                closest_file_paths[i] = file_path
                            else:
                                closest_file_paths.append(file_path)
    
    # Copy the closest slices to the output directory
    output_patient_dir = os.path.join(output_dir_path, patient_id)
    os.makedirs(output_patient_dir, exist_ok=True)
    
    for file_path in set(closest_file_paths):  # Use set to avoid duplicates
        if file_path:
            shutil.copy(file_path, output_patient_dir)
            print(f"Copied: {file_path} to {output_patient_dir}")

# Loop through each row in the DataFrame to process each patient
for index, row in df.iterrows():
    patient_id = str(row['Patient Number'])
    z_coords = [row['Bx Tip Z (MRI Coord)'], row['Bx Base Z (MRI Coord)']]
    find_and_copy_closest_slices(patient_id, z_coords)


Copied: /Users/arjunmoorthy/Desktop/targeted_bx_dicoms/Prostate-MRI-US-Biopsy-0001/06-28-2009-NA-MRI PROSTATE W WO CONTRAST-51743/11.000000-t2spcrstaxial oblProstate-90221/1-15.dcm to /Users/arjunmoorthy/Desktop/targeted_bx_dicoms/targeted_bx_slices_bothz/Prostate-MRI-US-Biopsy-0001
Copied: /Users/arjunmoorthy/Desktop/targeted_bx_dicoms/Prostate-MRI-US-Biopsy-0001/06-28-2009-NA-MRI PROSTATE W WO CONTRAST-51743/11.000000-t2spcrstaxial oblProstate-90221/1-23.dcm to /Users/arjunmoorthy/Desktop/targeted_bx_dicoms/targeted_bx_slices_bothz/Prostate-MRI-US-Biopsy-0001
Copied: /Users/arjunmoorthy/Desktop/targeted_bx_dicoms/Prostate-MRI-US-Biopsy-0001/06-28-2009-NA-MRI PROSTATE W WO CONTRAST-51743/11.000000-t2spcrstaxial oblProstate-90221/1-16.dcm to /Users/arjunmoorthy/Desktop/targeted_bx_dicoms/targeted_bx_slices_bothz/Prostate-MRI-US-Biopsy-0001
Copied: /Users/arjunmoorthy/Desktop/targeted_bx_dicoms/Prostate-MRI-US-Biopsy-0001/06-28-2009-NA-MRI PROSTATE W WO CONTRAST-51743/11.000000-t2spcrst

In [ ]:
import os
import numpy as np
import pandas as pd
import pydicom
import shutil

# Assuming dicom_dir_path and targeted_bx_path are defined
output_dir_path = os.path.join(dicom_dir_path, 'targeted_bx_slices_bothz')

# Ensure the output directory exists
os.makedirs(output_dir_path, exist_ok=True)

# Load the Excel file
df = pd.read_excel(targeted_bx_path)

closest_slices = {}  # Format: {z_coord: (file_path, [list of (x,y) coords], min_diff)}

def find_and_copy_closest_slices(patient_id, bx_points):
    patient_dicom_dir = os.path.join(dicom_dir_path, patient_id)
    # closest_slices = {}  # Format: {z_coord: (file_path, [list of (x,y) coords], min_diff)}

    # Walk through all subdirectories of the patient's DICOM directory
    for root, dirs, files in os.walk(patient_dicom_dir):
        for file in files:
            if file.endswith(".dcm"):
                file_path = os.path.join(root, file)
                ds = pydicom.dcmread(file_path)
                
                if 'ImagePositionPatient' in ds:
                    dicom_z_coord = ds.ImagePositionPatient[2]
                    for z_coord, xy_coords in bx_points.items():
                        diff = abs(dicom_z_coord - z_coord)
                        
                        if z_coord not in closest_slices or diff < closest_slices[z_coord][2]:
                            closest_slices[z_coord] = (file_path, [xy_coords], diff)
                        elif diff == closest_slices[z_coord][2]:
                            closest_slices[z_coord][1].append(xy_coords)  # Add xy_coords if the diff is equal
    
    # Copy the closest slices to the output directory
    output_patient_dir = os.path.join(output_dir_path, patient_id)
    os.makedirs(output_patient_dir, exist_ok=True)
    
    for z_coord, (file_path, xy_coords_list, _) in closest_slices.items():
        shutil.copy(file_path, output_patient_dir)
        print(f"Copied: {file_path} to {output_patient_dir} for z={z_coord} with xy points {xy_coords_list}")
    
    return closest_slices

# Convert DataFrame rows into a suitable structure for processing
for index, row in df.iterrows():
    patient_id = str(row['Patient Number'])
    bx_points = {
        row['Bx Tip Z (MRI Coord)']: (row['Bx Tip X (MRI Coord)'], row['Bx Tip Y (MRI Coord)']),
        row['Bx Base Z (MRI Coord)']: (row['Bx Base X (MRI Coord)'], row['Bx Base Y (MRI Coord)'])
    }
    find_and_copy_closest_slices(patient_id, bx_points)